## Coding Exercise #0706

### 1. Convolutional Neural Network (grayscale images):

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data            # MNIST hand written digits data!
import matplotlib.pyplot as plt
%matplotlib inline

#### 1.1. Download the MNIST data:

In [ ]:
# verbosity_saved = tf.logging.get_verbosity()                        # Save the current verbosity lebel if needed.
tf.logging.set_verbosity(tf.logging.ERROR)                            # Set the verbosity lebel high so that most warnings are ignored. 
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)         # Download the data.
type(mnist)                                                           # Check the type.

#### 1.2. Take a look at the dataset:

In [ ]:
print("Training data X shape: {}".format((mnist.train.images).shape))
print("Training data y shape: {}".format((mnist.train.labels).shape))
print("Training data cases: {}".format(mnist.train.num_examples))
print("\n")
print("Testing data X shape: {}".format((mnist.test.images).shape))
print("Testing data y shape: {}".format((mnist.test.labels).shape))
print("Testing data cases: {}".format(mnist.test.num_examples))

Visualization.

In [ ]:
i_image= 1                                                      # Image index. You can change it at will.
a_single_image = mnist.train.images[i_image].reshape(28,28)     #  Reshape as a 2D array.
plt.imshow(1-a_single_image, cmap='gist_gray')                  #  Display as grayscale image.
plt.show()

In [ ]:
# Check for the minimum and maximum pixel value.
# The data has been min-max-scaled already!
print("MIN : {}".format(a_single_image.min()))                 
print("MAX : {}".format(a_single_image.max())) 

#### 1.3. Define the hyperparameters and placeholders:

In [ ]:
batch_size = 8
n_epochs  = 10001
learn_rate = 0.0001
drop_prob = 0.5                                     # For the dropout layer.

In [ ]:
X_ph = tf.placeholder(tf.float32, [None, 784])      # 'None' means any number of rows (observations or batch_size)
y_ph = tf.placeholder(tf.float32,[None, 10])
drop_prob_ph = tf.placeholder(tf.float32)           # The drop probability at the dropout layer is a hyperparameter.
# X_ph must be reshaped in order to be fed into the conv2D function. 
# After reshaping, the shape becomes = [batch_size, in_height, in_width, in_channels].
# -1 allows flexible batch_size.
X_input_ph = tf.reshape(X_ph,[-1,28,28,1])             

#### 1.4. Define the Variables:

The configuration of the first convolution layer is as following:
- Kernel height = 5.
- Kernel width = 5.
- In_chanels = 1 (gray scale). 
- Out_channels = 32 (number of feature maps).

We need Variables with the folllowing shapes:
- Shape of the weight matrix = [kernel_height, kernel_width, in_channels, out_channels].
- Shape of the bias = [out_channels].

In [ ]:
# Variables are defined according to the specifications mentioned above.
W1 = tf.Variable(initial_value=tf.random_normal([5,5,1,32], mean=0, stddev=0.1))
b1 = tf.Variable(initial_value=tf.fill([32], 0.1)) 

The configuration of the second convolution layer is as following: 
- Kernel height = 5.
- Kernel width = 5.
- In_chanels = 32 (out_channels from the previous convolution layer). 
- Out_channels = 64 (number of feature maps).

Again, we need Variables with the folllowing shapes:
- Shape of the weight matrix = [kernel_height, kernel_width, in_channels, out_channels].
- Shape of the bias = [out_channels].

In [ ]:
# Variables are defined according to the specifications mentioned above.
W2 = tf.Variable(initial_value=tf.random_normal([5,5,32,64], mean=0, stddev=0.1))
b2 = tf.Variable(initial_value=tf.fill([64], 0.1)) 

We do the following considerations for the flattened fully connected layer:
- We will apply convolution twice with padding and there will be no image size reduction. 
- We will also apply max pooling twice with stride = 2 (vertically and horizontally). 
- At each max pooling with stride = 2, the image size is halved. Thus, (28/2)/2 = 7 will be the size (vertical and horizontal) of the resulting final image.   
- In the previous layer there were 64 output channels (feature maps). 
- Considering all these facts, there should be 7x7x64 = 3136 nodes in the flattened layer. 
- Finally, we will shrink the output from this layer to 1024.

In [ ]:
# Variables are defined according to the specifications mentioned above.
W3 = tf.Variable(initial_value=tf.random_normal([3136,1024], mean=0, stddev=0.1))
b3 = tf.Variable(initial_value=tf.fill([1024], 0.1))

We do the following considerations for the final output layer: 
- There are 1024 nodes to match with the output from the previous layer.
- We should shrink the output once more because there are 10 different labels (digits 0~9).

In [ ]:
# Variables are defined according to the specifications mentioned above.
W4 = tf.Variable(initial_value=tf.random_normal([1024,10], mean=0, stddev=0.1))
b4 = tf.Variable(initial_value=tf.fill([10], 0.1))    

#### 1.5. Define the deep learning model (CNN): 

Explanation of the arguments:
- padding = 'SAME' to apply a padding. padding = 'VALID' to apply no padding. 
- ksize = [1, kernel_height, kernel_width, 1]
- strides = [1, stride_vertical, stride_horizontal,1]

In [ ]:
# 1st Convolution layer.
y1 = tf.nn.conv2d(X_input_ph, W1, strides=[1, 1, 1, 1], padding='SAME') + b1
conv1 = tf.nn.relu(y1)                             # Apply the ReLu activation function. 
# 1st Pooling layer.
pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') 

In [ ]:
# 2nd Convolution layer.
y2 = tf.nn.conv2d(pool1, W2, strides=[1, 1, 1, 1], padding='SAME') + b2
conv2 = tf.nn.relu(y2)                            # Apply the ReLu activation function. 
# 2nd Pooling layer.
pool2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') 

In [ ]:
# Flattened full layer.
conv2_flattened = tf.reshape(pool2, [-1,3136])   # 7x7x64 = 3136.               
y3 = tf.matmul(conv2_flattened, W3) + b3    
full_layer = tf.nn.relu(y3)                      # Apply the ReLu activation function. 

In [ ]:
# Dropout layer.
dropout_layer = tf.nn.dropout(full_layer,rate = drop_prob_ph)

In [ ]:
# Output layer.                 
y_model = tf.matmul(dropout_layer, W4) + b4      # No activation function. Softmax at the output layer is optional. 

#### 1.6. Define the loss function and the optimizer:

In [ ]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_ph, logits=y_model))

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate = learn_rate) 

In [ ]:
train = optimizer.minimize(loss)

In [ ]:
init = tf.global_variables_initializer()

#### 1.7. Training and Testing:

In [ ]:
with tf.Session() as sess:
        sess.run(init)
        for i in range(n_epochs):
            batch_X, batch_y = mnist.train.next_batch(batch_size)                                            # Sample a batch!
            my_feed = {X_ph:batch_X, y_ph:batch_y, drop_prob_ph:drop_prob}
            sess.run(train, feed_dict = my_feed)
            if i % 500 == 0:            
                correct_predictions = tf.equal(tf.argmax(y_ph, axis=1), tf.argmax(y_model, axis=1))          # In argmax(), axis=1 means horizontal direction.
                accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))                          # Recast the Boolean as float32 first. Then calculate the mean.
                my_feed = {X_ph:mnist.test.images, y_ph:mnist.test.labels, drop_prob_ph:0.0}                 # No dropout for testing.
                accuracy_value = sess.run(accuracy, feed_dict = my_feed)                              
                print("Step = {}   ,   Accuracy = {:5.3f} \n".format(i, accuracy_value))